In [ ]:
import pandas as pd
import os
from PIL import Image
import torch
import timm
import pandas as pd
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import random_split, TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from torchvision.models import vit_b_16
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
from tqdm import tqdm
from sklearn.svm import LinearSVR
from scipy.stats import pearsonr
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# === Paths ===
drive_root = "/content/drive/MyDrive/BMI/Data"
csv_path = os.path.join(drive_root, "data.csv")
image_folder = os.path.join(drive_root, "Images")

# === Load CSV metadata ===
df = pd.read_csv(csv_path)

# === Filter out missing images upfront ===
df = df[df['name'].apply(lambda x: os.path.isfile(os.path.join(image_folder, x)))].reset_index(drop=True)
print(f"Valid samples after filtering missing images: {len(df)}")

# === Prepare file paths and labels ===
image_paths = [os.path.join(image_folder, fname) for fname in df['name']]
bmi_labels = df['bmi'].values.astype(np.float32)
gender_map = {'Male': 0, 'Female': 1}
gender_labels = df['gender'].map(gender_map).values.astype(np.int64)

# === Define transform (ViT style) ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# === Split indices for train/test (80/20) ===
num_samples = len(image_paths)
indices = np.arange(num_samples)
np.random.shuffle(indices)
split = int(0.8 * num_samples)
train_indices, test_indices = indices[:split], indices[split:]

# === Create DataLoader that yields indices ===
batch_size = 32

train_loader = DataLoader(train_indices, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_indices, batch_size=batch_size, shuffle=False, num_workers=2)

Valid samples after filtering missing images: 3962


In [ ]:
# === Function to load a batch of images and labels given indices ===
def load_batch(batch_indices):
    batch_images = []
    batch_bmis = []
    batch_genders = []
    for idx in batch_indices:
        img = Image.open(image_paths[idx]).convert('RGB')
        img = transform(img)
        batch_images.append(img)
        batch_bmis.append(bmi_labels[idx])
        batch_genders.append(gender_labels[idx])
    images_tensor = torch.stack(batch_images)
    bmi_tensor = torch.tensor(batch_bmis).unsqueeze(1)  # shape [batch, 1]
    gender_tensor = torch.tensor(batch_genders)
    return images_tensor, bmi_tensor, gender_tensor

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pretrained ViT
model = vit_b_16(weights='IMAGENET1K_V1')

# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze final transformer block and head
for name, param in model.encoder.layers[-1].named_parameters():
    param.requires_grad = True
for param in model.heads.parameters():
    param.requires_grad = True

# Replace classification head with regression head (output 1 BMI value)
model.heads = nn.Sequential(
    nn.Linear(model.heads.head.in_features, 1)
)

model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

Using device: cuda


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 189MB/s]


In [ ]:
num_epochs = 10
batch_size = 32

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_indices in train_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)
        images = images.to(device)
        bmis = bmis.to(device)
        # genders can be used later if you want, but not needed here

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/10], Loss: 521.5161
Epoch [2/10], Loss: 303.2960
Epoch [3/10], Loss: 193.5199
Epoch [4/10], Loss: 126.9845
Epoch [5/10], Loss: 93.7366
Epoch [6/10], Loss: 77.9454
Epoch [7/10], Loss: 71.7088
Epoch [8/10], Loss: 70.1345
Epoch [9/10], Loss: 67.3945
Epoch [10/10], Loss: 61.7492


In [ ]:
model.eval()

all_preds = []
all_labels = []
all_genders = []

with torch.no_grad():
    for batch_indices in test_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)

        images = images.to(device)
        outputs = model(images).cpu().squeeze().numpy()

        all_preds.extend(outputs)
        all_labels.extend(bmis.squeeze().numpy())
        all_genders.extend(genders.numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_genders = np.array(all_genders)

# Overall Pearson correlation
overall_r, _ = pearsonr(all_preds, all_labels)
print(f"Overall Pearson r: {overall_r:.4f}")

# Pearson r for males (gender 0)
male_mask = (all_genders == 0)
male_r, _ = pearsonr(all_preds[male_mask], all_labels[male_mask])
print(f"Male Pearson r: {male_r:.4f}")

# Pearson r for females (gender 1)
female_mask = (all_genders == 1)
female_r, _ = pearsonr(all_preds[female_mask], all_labels[female_mask])
print(f"Female Pearson r: {female_r:.4f}")

Overall Pearson r: 0.5151
Male Pearson r: 0.5363
Female Pearson r: 0.4956


In [ ]:
num_epochs = 5
batch_size = 32

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_indices in train_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)
        images = images.to(device)
        bmis = bmis.to(device)
        # genders can be used later if you want, but not needed here

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/5], Loss: 54.7205
Epoch [2/5], Loss: 49.3816
Epoch [3/5], Loss: 43.8568
Epoch [4/5], Loss: 39.5742
Epoch [5/5], Loss: 35.7136


In [ ]:
model.eval()

all_preds = []
all_labels = []
all_genders = []

with torch.no_grad():
    for batch_indices in test_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)

        images = images.to(device)
        outputs = model(images).cpu().squeeze().numpy()

        all_preds.extend(outputs)
        all_labels.extend(bmis.squeeze().numpy())
        all_genders.extend(genders.numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_genders = np.array(all_genders)

# Overall Pearson correlation
overall_r, _ = pearsonr(all_preds, all_labels)
print(f"Overall Pearson r: {overall_r:.4f}")

# Pearson r for males (gender 0)
male_mask = (all_genders == 0)
male_r, _ = pearsonr(all_preds[male_mask], all_labels[male_mask])
print(f"Male Pearson r: {male_r:.4f}")

# Pearson r for females (gender 1)
female_mask = (all_genders == 1)
female_r, _ = pearsonr(all_preds[female_mask], all_labels[female_mask])
print(f"Female Pearson r: {female_r:.4f}")

Overall Pearson r: 0.6052
Male Pearson r: 0.6535
Female Pearson r: 0.5523


In [ ]:
num_epochs = 10
batch_size = 32

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_indices in train_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)
        images = images.to(device)
        bmis = bmis.to(device)
        # genders can be used later if you want, but not needed here

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/10], Loss: 32.7932
Epoch [2/10], Loss: 31.5823
Epoch [3/10], Loss: 31.3000
Epoch [4/10], Loss: 25.6771
Epoch [5/10], Loss: 23.2953
Epoch [6/10], Loss: 22.5334
Epoch [7/10], Loss: 19.2481
Epoch [8/10], Loss: 17.9577
Epoch [9/10], Loss: 16.1079
Epoch [10/10], Loss: 16.0394


In [ ]:
model.eval()

all_preds = []
all_labels = []
all_genders = []

with torch.no_grad():
    for batch_indices in test_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)

        images = images.to(device)
        outputs = model(images).cpu().squeeze().numpy()

        all_preds.extend(outputs)
        all_labels.extend(bmis.squeeze().numpy())
        all_genders.extend(genders.numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_genders = np.array(all_genders)

# Overall Pearson correlation
overall_r, _ = pearsonr(all_preds, all_labels)
print(f"Overall Pearson r: {overall_r:.4f}")

# Pearson r for males (gender 0)
male_mask = (all_genders == 0)
male_r, _ = pearsonr(all_preds[male_mask], all_labels[male_mask])
print(f"Male Pearson r: {male_r:.4f}")s

# Pearson r for females (gender 1)
female_mask = (all_genders == 1)
female_r, _ = pearsonr(all_preds[female_mask], all_labels[female_mask])
print(f"Female Pearson r: {female_r:.4f}")

Overall Pearson r: 0.6076
Male Pearson r: 0.6687
Female Pearson r: 0.5374


In [ ]:
num_epochs = 5
batch_size = 32

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_indices in train_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)
        images = images.to(device)
        bmis = bmis.to(device)
        # genders can be used later if you want, but not needed here

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/5], Loss: 13.7384
Epoch [2/5], Loss: 12.2806
Epoch [3/5], Loss: 10.6522
Epoch [4/5], Loss: 9.7092
Epoch [5/5], Loss: 8.6710


In [ ]:
model.eval()

all_preds = []
all_labels = []
all_genders = []

with torch.no_grad():
    for batch_indices in test_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)

        images = images.to(device)
        outputs = model(images).cpu().squeeze().numpy()

        all_preds.extend(outputs)
        all_labels.extend(bmis.squeeze().numpy())
        all_genders.extend(genders.numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_genders = np.array(all_genders)

# Overall Pearson correlation
overall_r, _ = pearsonr(all_preds, all_labels)
print(f"Overall Pearson r: {overall_r:.4f}")

# Pearson r for males (gender 0)
male_mask = (all_genders == 0)
male_r, _ = pearsonr(all_preds[male_mask], all_labels[male_mask])
print(f"Male Pearson r: {male_r:.4f}")

# Pearson r for females (gender 1)
female_mask = (all_genders == 1)
female_r, _ = pearsonr(all_preds[female_mask], all_labels[female_mask])
print(f"Female Pearson r: {female_r:.4f}")

Overall Pearson r: 0.5907
Male Pearson r: 0.6508
Female Pearson r: 0.5164


In [ ]:
# Load pretrained ViT
model2 = vit_b_16(weights='IMAGENET1K_V1')

# === Step 1: Freeze all layers
for param in model2.parameters():
    param.requires_grad = False

# === Step 2: Unfreeze last two transformer blocks
for name, param in model2.encoder.layers[-1].named_parameters():
    param.requires_grad = True
for name, param in model2.encoder.layers[-2].named_parameters():
    param.requires_grad = True

# === Step 3: Replace and unfreeze the classification head for regression
model2.heads = nn.Sequential(
    nn.Linear(model2.heads.head.in_features, 1)
)
for param in model2.heads.parameters():
    param.requires_grad = True

# === Step 4: Set up training components
criterion = nn.MSELoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model2.parameters()),
    lr=1e-4
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [ ]:
# === Training loop for model2 ===
num_epochs = 15

for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0

    for batch_indices in train_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)
        images, bmis = images.to(device), bmis.to(device)

        optimizer.zero_grad()
        outputs = model2(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/15], Loss: 501.1352
Epoch [2/15], Loss: 301.6849
Epoch [3/15], Loss: 191.9251
Epoch [4/15], Loss: 126.9388
Epoch [5/15], Loss: 93.7092
Epoch [6/15], Loss: 78.0076
Epoch [7/15], Loss: 71.6919
Epoch [8/15], Loss: 70.0500
Epoch [9/15], Loss: 66.7818
Epoch [10/15], Loss: 61.1880
Epoch [11/15], Loss: 53.5123
Epoch [12/15], Loss: 48.8926
Epoch [13/15], Loss: 43.6410
Epoch [14/15], Loss: 40.3833
Epoch [15/15], Loss: 36.0550


In [ ]:
model2.eval()

all_preds = []
all_labels = []
all_genders = []

with torch.no_grad():
    for batch_indices in test_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)

        images = images.to(device)
        outputs = model2(images).cpu().squeeze().numpy()

        all_preds.extend(outputs)
        all_labels.extend(bmis.squeeze().numpy())
        all_genders.extend(genders.numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_genders = np.array(all_genders)

# Overall Pearson correlation
overall_r, _ = pearsonr(all_preds, all_labels)
print(f"Overall Pearson r: {overall_r:.4f}")

# Pearson r for males (gender 0)
male_mask = (all_genders == 0)
male_r, _ = pearsonr(all_preds[male_mask], all_labels[male_mask])
print(f"Male Pearson r: {male_r:.4f}")

# Pearson r for females (gender 1)
female_mask = (all_genders == 1)
female_r, _ = pearsonr(all_preds[female_mask], all_labels[female_mask])
print(f"Female Pearson r: {female_r:.4f}")

Overall Pearson r: 0.5679
Male Pearson r: 0.6091
Female Pearson r: 0.5177


In [ ]:
# Load pretrained ViT
model3 = vit_b_16(weights='IMAGENET1K_V1')

# Unfreeze last 3 transformer blocks + heads
for param in model3.parameters():
    param.requires_grad = False

for i in range(-3, 0):
    for param in model3.encoder.layers[i].parameters():
        param.requires_grad = True

for param in model3.heads.parameters():
    param.requires_grad = True

# === Step 3: Replace and unfreeze the classification head for regression
model3.heads = nn.Sequential(
    nn.Linear(model3.heads.head.in_features, 1)
)
for param in model3.heads.parameters():
    param.requires_grad = True

# === Step 4: Set up training components
criterion = nn.MSELoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model3.parameters()),
    lr=1e-4
)
scheduler = CosineAnnealingLR(optimizer, T_max=15)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [ ]:
num_epochs = 15

# === Training loop for model3 ===
for epoch in range(num_epochs):
    model3.train()
    running_loss = 0.0

    for batch_indices in train_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)
        images, bmis = images.to(device), bmis.to(device)

        optimizer.zero_grad()
        outputs = model3(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()  # Step the learning rate scheduler after each epoch

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/15], Loss: 487.4022
Epoch [2/15], Loss: 299.9130
Epoch [3/15], Loss: 193.6574
Epoch [4/15], Loss: 133.3260
Epoch [5/15], Loss: 96.9115
Epoch [6/15], Loss: 81.5277
Epoch [7/15], Loss: 74.6889
Epoch [8/15], Loss: 71.9249
Epoch [9/15], Loss: 71.6284
Epoch [10/15], Loss: 77.1209
Epoch [11/15], Loss: 69.3188
Epoch [12/15], Loss: 68.8281
Epoch [13/15], Loss: 67.4523
Epoch [14/15], Loss: 63.4685
Epoch [15/15], Loss: 62.0729


In [ ]:
model3.eval()

all_preds = []
all_labels = []
all_genders = []

with torch.no_grad():
    for batch_indices in test_loader:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)

        images = images.to(device)
        outputs = model3(images).cpu().squeeze().numpy()

        all_preds.extend(outputs)
        all_labels.extend(bmis.squeeze().numpy())
        all_genders.extend(genders.numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_genders = np.array(all_genders)

# Overall Pearson correlation
overall_r, _ = pearsonr(all_preds, all_labels)
print(f"Overall Pearson r: {overall_r:.4f}")

# Pearson r for males (gender 0)
male_mask = (all_genders == 0)
male_r, _ = pearsonr(all_preds[male_mask], all_labels[male_mask])
print(f"Male Pearson r: {male_r:.4f}")

# Pearson r for females (gender 1)
female_mask = (all_genders == 1)
female_r, _ = pearsonr(all_preds[female_mask], all_labels[female_mask])
print(f"Female Pearson r: {female_r:.4f}")

Overall Pearson r: 0.4427
Male Pearson r: 0.4594
Female Pearson r: 0.4301


In [ ]:
from torch.utils.data import Subset

# Split 20% of train_indices into validation
val_split = int(0.8 * len(train_indices))
final_train_indices = train_indices[:val_split]
val_indices = train_indices[val_split:]

# Create DataLoaders
train_loader2 = DataLoader(final_train_indices, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader2 = DataLoader(val_indices, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader2 = DataLoader(test_indices, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
print(f"Train samples: {len(final_train_indices)}")
print(f"Val samples: {len(val_indices)}")
print(f"Test samples: {len(test_indices)}")


Train samples: 2535
Val samples: 634
Test samples: 793


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model (using your vit_b_16 or timm vit_base_patch16_224)
model4 = vit_b_16(weights='IMAGENET1K_V1')
for param in model4.parameters():
    param.requires_grad = False

# Unfreeze last transformer block and head
for param in model4.encoder.layers[-1].parameters():
    param.requires_grad = True
for param in model4.heads.parameters():
    param.requires_grad = True

model4.heads = nn.Linear(model4.heads.head.in_features, 1)
model4.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model4.parameters()),
    lr=1e-5,
    weight_decay=1e-4
)

In [ ]:
# Training loop (with validation)
num_epochs = 50  # allow early stop
best_val_loss = float('inf')
patience = 7
counter = 0

for epoch in range(num_epochs):
    model4.train()
    train_loss = 0
    for batch_indices in train_loader2:
        batch_indices = batch_indices.numpy()
        images, bmis, _ = load_batch(batch_indices)
        images, bmis = images.to(device), bmis.to(device)

        optimizer.zero_grad()
        outputs = model4(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_loader2.dataset)

    # Validation
    model4.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_indices in val_loader2:
            batch_indices = batch_indices.numpy()
            images, bmis, _ = load_batch(batch_indices)
            images, bmis = images.to(device), bmis.to(device)
            outputs = model4(images)
            loss = criterion(outputs, bmis)
            val_loss += loss.item() * images.size(0)
    val_loss /= len(val_loader2.dataset)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model4.state_dict(), 'best_model.pth')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break

Epoch 1, Train Loss: 963.6217, Val Loss: 781.5673
Epoch 2, Train Loss: 664.4282, Val Loss: 587.8401
Epoch 3, Train Loss: 554.8544, Val Loss: 528.7000
Epoch 4, Train Loss: 512.1087, Val Loss: 496.9226
Epoch 5, Train Loss: 485.4034, Val Loss: 473.8698
Epoch 6, Train Loss: 464.4980, Val Loss: 454.7173
Epoch 7, Train Loss: 446.3294, Val Loss: 437.4102
Epoch 8, Train Loss: 429.7449, Val Loss: 421.3925
Epoch 9, Train Loss: 414.1753, Val Loss: 406.1804
Epoch 10, Train Loss: 399.3932, Val Loss: 391.7116
Epoch 11, Train Loss: 385.2322, Val Loss: 377.8428
Epoch 12, Train Loss: 371.6400, Val Loss: 364.3631
Epoch 13, Train Loss: 358.4650, Val Loss: 351.4430
Epoch 14, Train Loss: 345.7294, Val Loss: 338.8569
Epoch 15, Train Loss: 333.4128, Val Loss: 326.7060
Epoch 16, Train Loss: 321.5056, Val Loss: 315.0058
Epoch 17, Train Loss: 309.9238, Val Loss: 303.5757
Epoch 18, Train Loss: 298.6616, Val Loss: 292.4423
Epoch 19, Train Loss: 287.8139, Val Loss: 281.7512
Epoch 20, Train Loss: 277.2817, Val Loss

In [ ]:
import torchvision.models as models
from torchvision.models import resnet50, ResNet50_Weights

# Load a pre-trained ResNet50 model
model5 = models.resnet50(weights = ResNet50_Weights.DEFAULT)

# Replace the final fully connected layer for regression
model5.fc = nn.Sequential(
    nn.Linear(model5.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 1)  # Single output: predicted BMI
)

# Fine-tune entire network or freeze early layers
for param in model5.parameters():
    param.requires_grad = True  # Or freeze first few layers if you want

# Loss and optimizer
criterion = nn.SmoothL1Loss()  # Huber loss
optimizer = torch.optim.Adam(model5.parameters(), lr=1e-4)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 119MB/s]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model5 = model5.to(device)

In [ ]:
# === Training configuration ===
num_epochs = 10
patience = 7
best_val_loss = float('inf')
counter = 0



for epoch in range(num_epochs):
    # === Training ===
    model5.train()
    train_loss = 0
    for batch_indices in tqdm(train_loader2, desc=f"Epoch {epoch+1} - Training"):
        batch_indices = batch_indices.numpy()
        images, bmis, _ = load_batch(batch_indices)
        images, bmis = images.to(device), bmis.to(device)

        optimizer.zero_grad()
        outputs = model5(images)
        loss = criterion(outputs, bmis)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_loader2.dataset)

    # === Validation ===
    model5.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_indices in tqdm(val_loader2, desc=f"Epoch {epoch+1} - Validation"):
            batch_indices = batch_indices.numpy()
            images, bmis, _ = load_batch(batch_indices)
            images, bmis = images.to(device), bmis.to(device)

            outputs = model5(images)
            loss = criterion(outputs, bmis)
            val_loss += loss.item() * images.size(0)
    val_loss /= len(val_loader2.dataset)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # === Early stopping ===
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model5.state_dict(), 'best_model.pth')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

Epoch 1 - Validation: 100%|██████████| 20/20 [00:06<00:00,  3.25it/s]


Epoch 1, Train Loss: 2.9452, Val Loss: 4.2420


Epoch 2 - Validation: 100%|██████████| 20/20 [00:05<00:00,  3.54it/s]


Epoch 2, Train Loss: 2.8156, Val Loss: 4.1123


Epoch 3 - Validation: 100%|██████████| 20/20 [00:06<00:00,  3.30it/s]


...

Epoch 8, Train Loss: 2.1203, Val Loss: 4.1049


Epoch 9 - Validation: 100%|██████████| 20/20 [00:06<00:00,  3.30it/s]


Epoch 9, Train Loss: 2.0969, Val Loss: 4.0247


Epoch 10 - Validation: 100%|██████████| 20/20 [00:05<00:00,  3.59it/s]


Epoch 10, Train Loss: 2.1079, Val Loss: 3.9137


In [ ]:
model5.eval()

all_preds = []
all_labels = []
all_genders = []

with torch.no_grad():
    for batch_indices in test_loader2:
        batch_indices = batch_indices.numpy()
        images, bmis, genders = load_batch(batch_indices)

        images = images.to(device)
        outputs = model2(images).cpu().squeeze().numpy()

        all_preds.extend(outputs)
        all_labels.extend(bmis.squeeze().numpy())
        all_genders.extend(genders.numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_genders = np.array(all_genders)

# Overall Pearson correlation
overall_r, _ = pearsonr(all_preds, all_labels)
print(f"Overall Pearson r: {overall_r:.4f}")

# Pearson r for males (gender 0)
male_mask = (all_genders == 0)
male_r, _ = pearsonr(all_preds[male_mask], all_labels[male_mask])
print(f"Male Pearson r: {male_r:.4f}")

# Pearson r for females (gender 1)
female_mask = (all_genders == 1)
female_r, _ = pearsonr(all_preds[female_mask], all_labels[female_mask])
print(f"Female Pearson r: {female_r:.4f}")

Overall Pearson r: 0.5699
Male Pearson r: 0.6084
Female Pearson r: 0.5233


In [ ]:
pip install torch torchvision facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/2